# Grammatiche come generatori

In [ ]:
import os
os.environ["ANTLR4_JAR"] = "/home/federicobruzzoneplasma/Documents/FedericoBruzzone/master-courses/linguaggi-e-traduttori/lecture/jars/antlr-4.12.0-complete.jar"
from pprint import pprint as p

In [ ]:
from liblet import Grammar, Derivation, ProductionGraph

## Tipo 0

In [ ]:
# fig 2.3, pag. 14

grammar = """
Sentence -> Name | List End
List -> Name | Name , List
Name -> tom | dick | harry
, Name End -> and Name
"""

Tramite [liblet](https://liblet.readthedocs.io/) si può costruire una [grammatica](https://liblet.readthedocs.io/en/latest/api.html#liblet.grammar.Grammar) a partire da una descrizione testuale tramite [Grammar.from_string](https://liblet.readthedocs.io/en/latest/api.html#liblet.grammar.Grammar.from_string)

In [ ]:
G = Grammar.from_string(grammar, False)
p(G)

In [ ]:
# Elenco (numerato) delle produzioni

G.P

Fissata la grammatica, si può costruire una [derivazione](https://liblet.readthedocs.io/en/latest/api.html#liblet.grammar.Derivation) specificandone in passi, con [Derivation.step](https://liblet.readthedocs.io/en/latest/api.html#liblet.grammar.Derivation.step)

In [ ]:
# costruzione di una derivazione

d = Derivation(G)
d

In [ ]:
# i prossimi passi possibili

list(d.possible_steps())

In [ ]:
# ne applico uno

d = d.step(1, 0)
d

In [ ]:
# se volgio solo la forma sentenziale

d.sentential_form()

In [ ]:
# procedo con altri passi… 

d = d.step(3, 0)
d

In [ ]:
d = d.step(3, 2)
d

In [ ]:
d = (
  d.step(2, 4)
   .step(7, 3)
   .step(4, 0)
   .step(5, 2)
   .step(6, 4)
)

In [ ]:
d

In [ ]:
set(d.sentential_form()) <= G.T

Sebbene la rappresentazione testuale della derivazione sia piuttosto chiara, può aiutare averne una rappresentazione garfica, tramite un [ProductionGraph](https://liblet.readthedocs.io/en/latest/api.html#liblet.display.ProductionGraph)

In [ ]:
ProductionGraph(d)

## Tipo 1

### Monotone

In [ ]:
# pag 20

monotonic = """
Sentence -> Name | List
List -> EndName | Name , List
Name -> tom | dick | harry
, EndName -> and Name
"""

In [ ]:
G_monotonic = Grammar.from_string(monotonic, False)
G_monotonic

### Context-sentitive

In [ ]:
# pag 20

context_sensitive = """
Sentence -> Name | List
List -> EndName | Name Comma List
Name -> tom | dick | harry
Comma EndName -> and EndName
and EndName -> and Name
Comma -> ,
"""

In [ ]:
G_cs = Grammar.from_string(context_sensitive, False)
G_cs

In [ ]:
G_cs.P

In [ ]:
steps = (1, 0), (3, 0), (3, 2), (2, 4), (7, 3), (8, 3), (4, 0), (5, 2), (6, 4), (9, 1)

d = Derivation(G_cs).step(steps)

ProductionGraph(d)

## Tipo 2 (Context-free)

In [ ]:
# pag 23

context_free = """
Sentence -> Name | List and Name
List -> Name , List | Name
Name -> tom | dick | harry
"""

In [ ]:
steps = (1, 0), (2, 0), (3, 2), (4, 0), (5, 2), (6, 4)

G_cf = Grammar.from_string(context_free) # non c'è più il False

d = Derivation(G_cf).step(steps)

ProductionGraph(d) # finalmente un albero!

## Tipo 3 (Regolari)

In [ ]:
# fig. 2.14, pag. 31

regular = """
Sentence -> tom | dick | harry | List
List -> tom ListTail | dick ListTail | harry ListTail
ListTail -> , List | and tom | and dick | and henry
"""

regular_strict = """
Sentence -> tom | dick | harry | List
List -> tom ListTail | dick ListTail | harry ListTail
ListTail -> , List | and Tom | and Dick | and Henry
Tom -> tom
Dick -> dick
Henry -> henry
"""

In [ ]:
G_regular = Grammar.from_string(regular_strict)

steps = (3, 0), (4, 0), (7, 1), (5, 2), (10, 3), (13, 4)

d = Derivation(G_regular).step(steps)

ProductionGraph(d) # una "lista"

## Generare le parole

In [ ]:
from liblet import Queue

In [ ]:
# compie max_step passi di visita in ampiezza del grafo
# implicito delle derivazioni, restituendo l'eneco di derivazioni
# che contucono ad una forma sentenziale composta solo di terminali

def produce(G, max_steps = 1):
    Q = Queue([Derivation(G)])
    D = [] 
    step = 0
    while Q:
        if step > max_steps: break
        step += 1
        derivation = Q.dequeue()
        if set(derivation.sentential_form()) <= G.T: 
            D.append(derivation)
        else:
          for prod, pos in derivation.possible_steps():
              Q.enqueue(derivation.step(prod, pos))
    return D

In [ ]:
# la grammatica dell'esercizio facoltativo

G = Grammar.from_string("""
S -> a b c
S -> a S Q
b Q c -> b b c c
c Q -> Q c
""", False)

In [ ]:
# 10k passi per 42 derivazioni!

deriv = produce(G, 10_000)
len(deriv)

In [ ]:
# ma solo 5 parole distinte (come mai?)

words = set(''.join(d.sentential_form()) for d in deriv)
len(words)